## Multivariable Augmented

In [ ]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import json

# Supress Warning 
import warnings

# sklearn metrics and scaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

# Custom DL models
import importlib
from collections import defaultdict
from model import custom
from utils.callback import EarlyStopping, ModelCheckpoint

# Data Visualization
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# GPU memory setup to prevent crush
# gpu = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(gpu[0], True)

%matplotlib inline
# minus sign
plt.rcParams['axes.unicode_minus'] = False

# set save_path to bring data and save result
# we will save processed data here
save_path = './result/'
# # save results
# os.mkdir(save_path)

# ignore all warnings
warnings.filterwarnings('ignore')

# import custom models
importlib.reload(custom)

pd.set_option('mode.chained_assignment', None)

# columns of temp, export, consum
with open(save_path+'names.txt', 'r') as file:
    names = file.read()
names = json.loads(names)

# dictionary to store results
result = defaultdict(dict)

# Dataset
total = pd.read_csv(save_path+'preprocessed.csv', encoding='utf-8-sig')
consum = pd.read_csv(save_path+'original.csv', encoding='utf-8-sig')

# Functions

## Data Augmentation
def aug(data):
    d = data.copy()
    col_name = list(set(d.columns)-set(['period']))
    d[col_name] = d[col_name].values.astype('float32')
    d['period'] = pd.to_datetime(d['period'])
    # dataframe to series
    d = d.squeeze()
    # TimeIndex
    d = d.set_index('period')
    # upsampling by Day
    upsampled = d.resample('D')
    # fillna with Spline interpolation
    interpolated = upsampled.interpolate(method='spline', order=2)
    interpolated = interpolated.dropna()
    return interpolated

## Apply Data Augmentation
def get_data(data):
    # Nationwide data
    temp = data.copy()
    # state-level 
    state = list(set(temp["region"]))
    ## Data Augmentation
    emp = []
    for i in state:
        df = temp[(temp["region"]==i) & (temp["contract"]=='total')]
        df = df[df.columns[~df.columns.isin(["Unnamed: 0", "region", "contract",'Unnamed: 0_x', 'Unnamed: 0_y'])]]
        df = aug(df.copy())
        df = df.reset_index()
        df['region'] = i
        df['contract'] = 'total'
        emp.append(df)
    return pd.concat(emp, ignore_index=True)

## Data Slicing
def slicing(data):
    n = len(data)
    train_df = data[0:int(n*0.7), :]
    test_df = data[int(n*0.7):, :]
    return train_df, test_df

## Visualize Loss
def his(data, name, location):
    plt.plot(data.history['loss'], label='train')
    plt.plot(data.history['val_loss'], label='val')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title("===="+location+"===="+name+"====")
    plt.legend()
    plt.savefig(save_path + 'multi_aug' + location + name + '_loss.png', dpi=700)
    plt.show()

## Evaluation Metrics
RMSE = lambda x, y: mean_squared_error(x, y, squared=False)
MAPE = mean_absolute_percentage_error

def evaluation(label, outputs, label_width):
    return {
        "RMSE": [RMSE(label[:, i], outputs[:, i]) for i in range(label_width)],
        "MAPE": [MAPE(label[:, i], outputs[:, i]) for i in range(label_width)],
    }

## Extract Forecasting for Original dataset (i.e. monthly dataset)
def extract(data, augmented, input_width, train=False):
    df = pd.DataFrame(data)
    augmented = augmented.reset_index()
    months = consum['period'].values
    daily = augmented['period'].copy()
    n = len(daily)
    if train:
        df['period'] = daily[input_width:int(n*0.7)].values
    else:
        df['period'] = daily[int(n*0.7)+input_width:].values
    df = df.loc[df['period'].isin(months)]
    df = df.drop(columns=['period'])
    return df.values.astype('float32')

## nested Dict to Dataframe
def dict2df(dict):
    reformed_dict = {}
    for outerKey, innerDict in dict.items():
        for innerKey, values in innerDict.items():
            reformed_dict[(outerKey, innerKey)] = values
    dict = pd.DataFrame(reformed_dict)
    return dict 

## Modeling DL Algorithms 
def Modeling(dataset, model_name, config, label, location):  
    # config
    input_width = config["input_width"]
    label_width = config["label_width"]
    batch_size = config["batch_size"]
    epochs = config["epochs"]
    vb = config["vb"]
    units = config["units"]
    drop = config["drop"]
    
    n_features = 3

    dataset = dataset[(dataset["region"]==location) & (dataset["contract"]==label)]
    dataset = dataset[["period", "usage(kWh)", "export_value", "import_value"]]
    col_name = list(set(dataset.columns)-set(['period']))
    dataset[col_name] = dataset[col_name].values.astype('float32')
    dataset['period'] = pd.to_datetime(dataset['period'])

    # TimeIndex
    dataset = dataset.set_index('period')
    values = dataset.values

    # Data Slicing
    train_set, test_set = slicing(data=values)

    # Normalization 
    scaler = MinMaxScaler(feature_range=(0, 1))    
    train_set = scaler.fit_transform(train_set)
    test_set = scaler.transform(test_set)

    # reshape into X=t and Y=t+timestep
    train_set = custom.generator(train_set, input_width, label_width)
    test_set = custom.generator(test_set, input_width, label_width)
 
    train_set = train_set.values
    test_set = test_set.values

    # split into input and outputs
    n_obs = input_width * n_features
    trainX, trainY = train_set[:, :n_obs], train_set[:, -n_features]
    testX, testY = test_set[:, :n_obs], test_set[:, -n_features]
    
    # reshape output to be [samples, timesteps (lag), features]
    trainX = trainX.reshape((trainX.shape[0], input_width, n_features))
    testX = testX.reshape((testX.shape[0], input_width, n_features))

    # create and fit model
    models = custom.get_model(model_name, units=units, input_width = input_width, 
                            label_width = label_width, n_features=n_features, dropout = drop)
    history = models.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, 
                    validation_data = (testX, testY), verbose=vb[0], 
                    # callbacks=[
                    #     EarlyStopping(patience=30, verbose=vb[0], min_epoch=100),
                    #     ModelCheckpoint(f"{save_path}_multi_aug_{model_name}_{location}.h5", verbose=vb[0], min_epoch=100,
                    #     save_best_only=True, save_weights_only=True)],
                    shuffle=False)

    # # model summary
    # print(models.summary())
    
    # # plot history
    # his(history, model_name, location)

    # make predictions
    trainPredict = models.predict(trainX)
    testPredict = models.predict(testX)

    trainY = trainY.reshape((len(trainY), 1))
    testY = testY.reshape((len(testY), 1))

    ## Inverse Transform
    def inverse(Predict, X, Y):
        # reshape for inverse scaling
        X = X.reshape((X.shape[0], input_width*n_features)) 
        if model_name == "linear":
            prd = np.concatenate((Predict[:, 0, :], X[:, -(n_features-1):]), axis=1)
        else:
            prd = np.concatenate((Predict[:, :, 0], X[:, -(n_features-1):]), axis=1)
        trg = np.concatenate((Y, X[:, -(n_features-1):]), axis=1)
        # inverse scaling prediction
        inverse_prd = scaler.inverse_transform(prd)
        # inverse scaling target
        inverse_trg = scaler.inverse_transform(trg)
        return inverse_prd, inverse_trg
            
    train_prd, train_trg = inverse(trainPredict, trainX, trainY)
    test_prd, test_trg = inverse(testPredict, testX, testY)

    # calculate evaluation metrics
    testEval = evaluation(testY, testPredict[:, :, 0], label_width=label_width)

    # extract monthly result
    train_prd = extract(train_prd, dataset, input_width, train=True)
    test_prd = extract(test_prd, dataset, input_width)
    train_trg = extract(train_trg, dataset, input_width, train=True)
    test_trg = extract(test_trg, dataset, input_width)

    # # shift train predictions for plotting
    # trainPlot = np.empty_like(dataset)
    # trainPlot[:, :] = np.nan
    # trainPlot[input_width:len(train_prd)+input_width, 0] = train_prd[:, 0]
    # # shift test predictions for plotting
    # testPlot = np.empty_like(dataset)
    # testPlot[:, :] = np.nan
    # testPlot[len(train_prd)+input_width:len(train_prd)+len(test_prd)+input_width, 0] = test_prd[:, 0]
    # # unshifted baseline
    # baseline  = np.empty_like(dataset)
    # baseline[:, :] = np.nan
    # baseline[:len(train_trg), 0] = train_trg[:, 0]
    # baseline[len(train_trg):len(train_trg)+len(test_trg), 0] = test_trg[:, 0]
    # # plot baseline and predictions
    # plt.plot(baseline)
    # plt.plot(trainPlot)
    # plt.plot(testPlot)  
    # plt.title('multi_aug'+model_name+location+'_trg_prd')
    # plt.savefig(f'{save_path}multi_aug_{model_name}_{location}_trg_prd.png', dpi=500)
    # plt.show()

    result[model_name][location] = {}
    result[model_name][location]['eval'] = testEval
    result[model_name][location]['targ'] = test_trg
    result[model_name][location]['prds'] = test_prd

    # print(location)
    # print(result[model_name][location]['eval']['RMSE'])
    # print(result[model_name][location]['eval']['MAPE'])
    return result

## Main
def main(dataset, model_name, label, location="all", config=None):
    # Nationwide data
    exm = dataset.copy()
    bymean = exm[names['temp']].copy()
    bymean = bymean.groupby(['period']).mean().reset_index()
    bysum = exm[set(names['consum']+names['export'])].copy()
    bysum = bysum.groupby(['period', 'contract']).sum().reset_index()
    bysum["region"] = "KOR"
    bysum = bysum.merge(bymean, how='outer', on='period').reset_index(drop=True)
    dataset  = pd.concat([exm, bysum], ignore_index=True)
    dataset = dataset[dataset["region"] != 'Sejong']
    # state-level 
    state = list(set(dataset["region"])-set(["KOR"]))
    dataset = get_data(dataset)
    if location=="all":
        for i in state:
            result = Modeling(dataset = dataset, model_name=model_name, 
                            config=config, label=label, location=i)
    # nation-level
    else:
        result = Modeling(dataset = dataset, model_name=model_name, 
                        config=config, label=label, location=location)

    result = dict2df(result)
    return result

In [ ]:
## Single-step model
np.random.seed(123)  #123
tf.random.set_seed(123) #123

# learning_rate = 1e-6
# decay = None

units = 128 # 128 dimension 
input_width = 30 # input sequence (number of lag days) 
label_width = 1 # forecast sequence (number of output days)
batch_size = 16 #8
epochs = 100 
drop = 0.1
vb = [0, 1, 2]

config = {'units': units, 'input_width': input_width, 
        'label_width': label_width, 'batch_size': batch_size, 
        'epochs': epochs, 'drop': drop, 'vb': vb}

In [ ]:
## National/State-level
# n_lin: national
# all_lin: all states
n_lin = main(dataset=total, model_name="linear", label= "total", location="KOR", config=config)
all_lin = main(dataset=total, model_name="linear", label= "total", location="all", config=config)
n_rnn =  main(dataset=total, model_name="rnn", label= "total", location="KOR", config=config)
all_rnn = main(dataset=total, model_name="rnn", label= "total", location="all", config=config)
n_lstm = main(dataset=total, model_name="lstm", label= "total", location="KOR", config=config)
all_lstm = main(dataset=total, model_name="lstm", label= "total", location="all", config=config)
n_cnnlstm = main(dataset=total, model_name="cnn-lstm", label= "total", location="KOR", config=config)
all_cnnlstm = main(dataset=total, model_name="cnn-lstm", label= "total", location="all", config=config)

In [ ]:
# save result to csv
# all_cnnlstm has all results of previous training
all_cnnlstm.to_csv(save_path+'multi_aug_result.csv', encoding='utf-8-sig')